In [3]:
"""
================================================================================
PASO 10: MEJORA Y VALIDACIÓN AVANZADA DE MODELOS
================================================================================

Proyecto: Predicción de Síndrome de Ovario Poliquístico (SOP)
Institución: Clúster de Ingeniería Biomédica del Estado de Jalisco
Fecha: 2 noviembre 2025
Autor: Usuario (QFB) + Claude (Asistente)

OBJETIVO:
Refinar y validar exhaustivamente los modelos del Paso 9

CONTENIDO:
FASE 1 - MÉTRICAS CLÍNICAS:
  - Sensibilidad, Especificidad, PPV, NPV
  - Matrices de confusión extendidas
  
FASE 2 - EXPLICABILIDAD:
  - SHAP values (XGBoost y Random Forest)
  - Interpretación top 10 features
  
FASE 3 - ENSEMBLE:
  - Voting Classifier (RF + XGBoost)
  - Comparación vs modelos individuales
  
FASE 4 - OPTIMIZACIÓN RFE:
  - Reducir features de 40 a 20
  - Mantener performance
  
FASE 5 - VALIDACIÓN REPETIDA:
  - Repeated Stratified K-Fold (5×3)
  - Intervalos de confianza

REQUISITOS PREVIOS:
- Ejecutar PASO_09_Modelado_ML_Completo.py
- Tener datasets en mismo directorio
- Instalar: pip install shap --break-system-packages

================================================================================
"""

# =============================================================================
# SECCIÓN 0: IMPORTS Y CONFIGURACIÓN
# =============================================================================

print("="*80)
print("PASO 10: MEJORA Y VALIDACIÓN AVANZADA DE MODELOS")
print("="*80)
print()

import warnings
warnings.filterwarnings('ignore')

# Librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Scikit-learn
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold,
    RepeatedStratifiedKFold,
    cross_val_score,
    cross_validate,
    GridSearchCV
)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix,
    classification_report, make_scorer
)

from imblearn.over_sampling import SMOTE

# SHAP para explicabilidad
try:
    import shap
    SHAP_AVAILABLE = True
    print("✓ SHAP disponible")
except ImportError:
    SHAP_AVAILABLE = False
    print("⚠️ SHAP no disponible. Instalar: pip install shap --break-system-packages")
    print("   Continuando sin SHAP...")

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Semilla global
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Imports completados")
print()

# =============================================================================
# SECCIÓN 1: CARGAR DATOS Y RECREAR MODELOS
# =============================================================================

print("="*80)
print("SECCIÓN 1: CARGA DE DATOS Y PREPARACIÓN")
print("="*80)
print()

# AJUSTAR RUTAS
PATH_DATASET_A = '../documentos_generados/PCOS_data_transformado.csv'
PATH_DATASET_B = 'PCOS_data_FINAL_sin_multicolinealidad.csv'
TARGET_COL = 'SOP (S/N)'
TEST_SIZE = 0.20

print("Cargando datasets...")
df_trees = pd.read_csv(PATH_DATASET_A)
df_logit = pd.read_csv(PATH_DATASET_B)

# LIMPIAR VALORES CON FORMATO '[5E-1]' o similares
print("Limpiando formato de datos...")

def limpiar_columna(col):
    """Limpia valores con formato [5E-1] convirtiéndolos a float"""
    if col.dtype == 'object':
        # Remover corchetes y convertir
        col = col.astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False)
        try:
            col = pd.to_numeric(col, errors='coerce')
        except:
            pass
    return col

# Aplicar limpieza a todas las columnas numéricas
for col in df_trees.columns:
    if col != TARGET_COL:
        df_trees[col] = limpiar_columna(df_trees[col])

for col in df_logit.columns:
    if col != TARGET_COL:
        df_logit[col] = limpiar_columna(df_logit[col])

# Verificar si quedaron NaN después de limpieza
nans_trees = df_trees.isnull().sum().sum()
nans_logit = df_logit.isnull().sum().sum()

if nans_trees > 0 or nans_logit > 0:
    print(f"⚠️ NaN detectados después de limpieza:")
    print(f"   Dataset A: {nans_trees} NaN")
    print(f"   Dataset B: {nans_logit} NaN")
    print("   Rellenando con mediana por columna...")
    
    # Rellenar con mediana
    for col in df_trees.select_dtypes(include=[np.number]).columns:
        if df_trees[col].isnull().any():
            df_trees[col].fillna(df_trees[col].median(), inplace=True)
    
    for col in df_logit.select_dtypes(include=[np.number]).columns:
        if df_logit[col].isnull().any():
            df_logit[col].fillna(df_logit[col].median(), inplace=True)

# Verificar tipos de datos
print("Verificando tipos de datos...")
non_numeric_trees = df_trees.drop(TARGET_COL, axis=1).select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric_logit = df_logit.drop(TARGET_COL, axis=1).select_dtypes(exclude=[np.number]).columns.tolist()

if non_numeric_trees:
    print(f"⚠️ Columnas no numéricas en Dataset A: {non_numeric_trees}")
    print("   Convirtiendo a numérico...")
    for col in non_numeric_trees:
        df_trees[col] = pd.to_numeric(df_trees[col], errors='coerce')
        if df_trees[col].isnull().any():
            df_trees[col].fillna(df_trees[col].median(), inplace=True)

if non_numeric_logit:
    print(f"⚠️ Columnas no numéricas en Dataset B: {non_numeric_logit}")
    print("   Convirtiendo a numérico...")
    for col in non_numeric_logit:
        df_logit[col] = pd.to_numeric(df_logit[col], errors='coerce')
        if df_logit[col].isnull().any():
            df_logit[col].fillna(df_logit[col].median(), inplace=True)

print(f"✓ Dataset A limpio: {df_trees.shape}")
print(f"✓ Dataset B limpio: {df_logit.shape}")
print()

# Train/Test Split
print("Preparando datos...")

# Dataset A (árboles)
X_trees = df_trees.drop(TARGET_COL, axis=1)
y_trees = df_trees[TARGET_COL]
X_train_trees, X_test_trees, y_train_trees, y_test_trees = train_test_split(
    X_trees, y_trees, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_trees
)

# Dataset B (logística)
X_logit = df_logit.drop(TARGET_COL, axis=1)
y_logit = df_logit[TARGET_COL]
X_train_logit, X_test_logit, y_train_logit, y_test_logit = train_test_split(
    X_logit, y_logit, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_logit
)

# SMOTE
smote_trees = SMOTE(random_state=RANDOM_STATE)
X_train_trees_balanced, y_train_trees_balanced = smote_trees.fit_resample(
    X_train_trees, y_train_trees
)

smote_logit = SMOTE(random_state=RANDOM_STATE)
X_train_logit_balanced, y_train_logit_balanced = smote_logit.fit_resample(
    X_train_logit, y_train_logit
)

# Escalamiento
scaler_logit = StandardScaler()
X_train_logit_scaled = pd.DataFrame(
    scaler_logit.fit_transform(X_train_logit_balanced),
    columns=X_train_logit.columns
)
X_test_logit_scaled = pd.DataFrame(
    scaler_logit.transform(X_test_logit),
    columns=X_test_logit.columns
)

scaler_knn = StandardScaler()
X_train_trees_scaled = pd.DataFrame(
    scaler_knn.fit_transform(X_train_trees_balanced),
    columns=X_train_trees.columns
)
X_test_trees_scaled = pd.DataFrame(
    scaler_knn.transform(X_test_trees),
    columns=X_test_trees.columns
)

print("✓ Datos preparados (split, SMOTE, escalamiento)")
print()

# =============================================================================
# FASE 1: MÉTRICAS CLÍNICAS EXTENDIDAS
# =============================================================================

print("="*80)
print("FASE 1: MÉTRICAS CLÍNICAS EXTENDIDAS")
print("="*80)
print()

def calcular_metricas_clinicas(y_true, y_pred, model_name):
    """Calcula métricas clínicas detalladas"""
    
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Métricas básicas
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Métricas clínicas
    sensibilidad = tp / (tp + fn) if (tp + fn) > 0 else 0  # = Recall
    especificidad = tn / (tn + fp) if (tn + fp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Positive Predictive Value = Precision
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
    
    # Likelihood Ratios
    lr_pos = sensibilidad / (1 - especificidad) if especificidad < 1 else np.inf
    lr_neg = (1 - sensibilidad) / especificidad if especificidad > 0 else np.inf
    
    return {
        'Model': model_name,
        'TN': int(tn), 'FP': int(fp), 'FN': int(fn), 'TP': int(tp),
        'Accuracy': accuracy,
        'Precision (PPV)': ppv,
        'Recall (Sensibilidad)': sensibilidad,
        'Especificidad': especificidad,
        'F1-Score': f1,
        'NPV': npv,
        'LR+': lr_pos,
        'LR-': lr_neg
    }

# Entrenar modelos rápidos (sin Grid Search por velocidad)
print("Entrenando modelos para evaluación...")
print()

# 1. Logistic Regression
lr_model = LogisticRegression(C=0.1, max_iter=1000, random_state=RANDOM_STATE)
lr_model.fit(X_train_logit_scaled, y_train_logit_balanced)
y_pred_lr = lr_model.predict(X_test_logit_scaled)

# 2. Random Forest (con 40 features - del Paso 9)
# Recrear RFE para obtener las 40 features
rf_estimator = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rfe = RFE(estimator=rf_estimator, n_features_to_select=40, step=1)
rfe.fit(X_train_trees_balanced, y_train_trees_balanced)
X_train_rf = rfe.transform(X_train_trees_balanced)
X_test_rf = rfe.transform(X_test_trees)

rf_model = RandomForestClassifier(
    n_estimators=200, max_depth=10, min_samples_split=5, 
    min_samples_leaf=1, random_state=RANDOM_STATE
)
rf_model.fit(X_train_rf, y_train_trees_balanced)
y_pred_rf = rf_model.predict(X_test_rf)

# 3. XGBoost
xgb_model = xgb.XGBClassifier(
    learning_rate=0.1, n_estimators=100, max_depth=5,
    subsample=0.8, random_state=RANDOM_STATE, eval_metric='logloss'
)
xgb_model.fit(X_train_trees_balanced, y_train_trees_balanced)
y_pred_xgb = xgb_model.predict(X_test_trees)

# 4. KNN
knn_model = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='manhattan')
knn_model.fit(X_train_trees_scaled, y_train_trees_balanced)
y_pred_knn = knn_model.predict(X_test_trees_scaled)

print("✓ Modelos entrenados")
print()

# Calcular métricas clínicas
print("MÉTRICAS CLÍNICAS EXTENDIDAS:")
print()

metricas_clinicas = []
metricas_clinicas.append(calcular_metricas_clinicas(y_test_logit, y_pred_lr, 'Logistic Regression'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_rf, 'Random Forest'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_xgb, 'XGBoost'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_knn, 'KNN'))

df_clinicas = pd.DataFrame(metricas_clinicas)

print(df_clinicas.to_string(index=False))
print()

# Guardar
df_clinicas.to_csv('metricas_clinicas_extendidas.csv', index=False)
print("✓ Guardado: metricas_clinicas_extendidas.csv")
print()

# Interpretación clínica
print("INTERPRETACIÓN CLÍNICA:")
print()
print("Sensibilidad (Recall): % de SOP reales detectados")
print("  → Alto = Menos falsos negativos (no miss casos SOP)")
print()
print("Especificidad: % de No-SOP correctamente identificados")
print("  → Alto = Menos falsos positivos (menos alarmas falsas)")
print()
print("PPV (Precision): Si predice SOP, ¿cuál es probabilidad real?")
print("  → Alto = Predicción SOP confiable")
print()
print("NPV: Si predice No-SOP, ¿cuál es probabilidad real?")
print("  → Alto = Predicción negativa confiable")
print()

# =============================================================================
# FASE 2: EXPLICABILIDAD CON SHAP
# =============================================================================

print("="*80)
print("FASE 2: EXPLICABILIDAD CON SHAP VALUES")
print("="*80)
print()

if not SHAP_AVAILABLE:
    print("⚠️ SHAP no disponible. Saltando esta sección.")
    print("   Para instalar: pip install shap --break-system-packages")
    print()
else:
    try:
        print("Calculando SHAP values para XGBoost...")
        print("(Esto puede tardar 2-3 minutos)")
        print()
        
        # SHAP para XGBoost (más rápido)
        explainer_xgb = shap.TreeExplainer(xgb_model)
        shap_values_xgb = explainer_xgb.shap_values(X_test_trees)
        
        # Summary plot XGBoost
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_xgb, X_test_trees, plot_type="bar", show=False, max_display=15)
        plt.title('XGBoost - Top 15 Features (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_xgboost_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Guardado: shap_xgboost_summary.png")
        
        # Beeswarm plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_xgb, X_test_trees, show=False, max_display=15)
        plt.title('XGBoost - Distribución de Impacto (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_xgboost_beeswarm.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Guardado: shap_xgboost_beeswarm.png")
        
        # Extraer valores promedio
        shap_importance = pd.DataFrame({
            'Feature': X_test_trees.columns,
            'SHAP_Mean_Abs': np.abs(shap_values_xgb).mean(axis=0)
        }).sort_values('SHAP_Mean_Abs', ascending=False)
        
        shap_importance.to_csv('shap_importance_xgboost.csv', index=False)
        print("✓ Guardado: shap_importance_xgboost.csv")
        print()
        
        print("TOP 10 FEATURES (SHAP):")
        print(shap_importance.head(10).to_string(index=False))
        print()
        
        # SHAP para Random Forest (opcional, más lento)
        print("Calculando SHAP values para Random Forest...")
        print("(Esto puede tardar 3-5 minutos)")
        print()
        
        explainer_rf = shap.TreeExplainer(rf_model)
        shap_values_rf = explainer_rf.shap_values(X_test_rf)
        
        # Si es clasificación binaria, tomar clase positiva
        if isinstance(shap_values_rf, list):
            shap_values_rf = shap_values_rf[1]
        
        # Summary plot RF
        selected_features = X_train_trees_balanced.columns[rfe.support_].tolist()
        X_test_rf_df = pd.DataFrame(X_test_rf, columns=selected_features)
        
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_rf, X_test_rf_df, plot_type="bar", show=False, max_display=15)
        plt.title('Random Forest - Top 15 Features (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_rf_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Guardado: shap_rf_summary.png")
        print()
        
    except Exception as e:
        print(f"⚠️ Error en SHAP: {str(e)}")
        print("   Continuando sin análisis SHAP...")
        print()
        SHAP_AVAILABLE = False

print("✓ FASE 2 completada")
print()

# =============================================================================
# FASE 3: ENSEMBLE (VOTING CLASSIFIER)
# =============================================================================

print("="*80)
print("FASE 3: ENSEMBLE - VOTING CLASSIFIER")
print("="*80)
print()

print("Creando ensemble RF + XGBoost...")
print()

# Ensemble con los mejores modelos
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model)
    ],
    voting='soft',  # Usa probabilidades
    weights=[1, 1]  # Mismo peso
)

# Entrenar ensemble (en los datos transformados de RF)
ensemble.fit(X_train_rf, y_train_trees_balanced)
y_pred_ensemble = ensemble.predict(X_test_rf)
y_pred_proba_ensemble = ensemble.predict_proba(X_test_rf)[:, 1]

# Métricas ensemble
accuracy_ensemble = accuracy_score(y_test_trees, y_pred_ensemble)
precision_ensemble = precision_score(y_test_trees, y_pred_ensemble)
recall_ensemble = recall_score(y_test_trees, y_pred_ensemble)
f1_ensemble = f1_score(y_test_trees, y_pred_ensemble)
auc_ensemble = roc_auc_score(y_test_trees, y_pred_proba_ensemble)

print("RESULTADOS ENSEMBLE:")
print(f"  Accuracy:  {accuracy_ensemble:.4f}")
print(f"  Precision: {precision_ensemble:.4f}")
print(f"  Recall:    {recall_ensemble:.4f}")
print(f"  F1-Score:  {f1_ensemble:.4f}")
print(f"  ROC-AUC:   {auc_ensemble:.4f}")
print()

# Comparación con modelos individuales
print("COMPARACIÓN:")
print()
comparison_ensemble = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'Ensemble (RF+XGB)'],
    'Accuracy': [
        accuracy_score(y_test_trees, y_pred_rf),
        accuracy_score(y_test_trees, y_pred_xgb),
        accuracy_ensemble
    ],
    'Precision': [
        precision_score(y_test_trees, y_pred_rf),
        precision_score(y_test_trees, y_pred_xgb),
        precision_ensemble
    ],
    'Recall': [
        recall_score(y_test_trees, y_pred_rf),
        recall_score(y_test_trees, y_pred_xgb),
        recall_ensemble
    ],
    'F1-Score': [
        f1_score(y_test_trees, y_pred_rf),
        f1_score(y_test_trees, y_pred_xgb),
        f1_ensemble
    ],
    'ROC-AUC': [
        roc_auc_score(y_test_trees, rf_model.predict_proba(X_test_rf)[:, 1]),
        roc_auc_score(y_test_trees, xgb_model.predict_proba(X_test_trees)[:, 1]),
        auc_ensemble
    ]
})

print(comparison_ensemble.to_string(index=False))
print()

comparison_ensemble.to_csv('ensemble_comparison.csv', index=False)
print("✓ Guardado: ensemble_comparison.csv")
print()

# Métricas clínicas del ensemble
metricas_ensemble = calcular_metricas_clinicas(y_test_trees, y_pred_ensemble, 'Ensemble (RF+XGB)')
print("MÉTRICAS CLÍNICAS ENSEMBLE:")
print(pd.DataFrame([metricas_ensemble]).to_string(index=False))
print()

# =============================================================================
# FASE 4: OPTIMIZACIÓN RFE (REDUCIR A 20 FEATURES)
# =============================================================================

print("="*80)
print("FASE 4: OPTIMIZACIÓN RFE - REDUCIR A 20 FEATURES")
print("="*80)
print()

print("Probando RFE con diferentes números de features...")
print()

n_features_range = [10, 15, 20, 25, 30]
rfe_optimization_results = []

for n_features in n_features_range:
    rf_estimator_opt = RandomForestClassifier(
        n_estimators=200, max_depth=10, min_samples_split=5,
        random_state=RANDOM_STATE, n_jobs=-1
    )
    
    rfe_opt = RFE(estimator=rf_estimator_opt, n_features_to_select=n_features, step=1)
    rfe_opt.fit(X_train_trees_balanced, y_train_trees_balanced)
    
    X_train_rfe_opt = rfe_opt.transform(X_train_trees_balanced)
    X_test_rfe_opt = rfe_opt.transform(X_test_trees)
    
    # Cross-validation
    cv_scores = cross_val_score(
        rf_estimator_opt, X_train_rfe_opt, y_train_trees_balanced,
        cv=5, scoring='f1'
    )
    
    # Test performance
    rf_estimator_opt.fit(X_train_rfe_opt, y_train_trees_balanced)
    y_pred_opt = rf_estimator_opt.predict(X_test_rfe_opt)
    
    test_f1 = f1_score(y_test_trees, y_pred_opt)
    test_acc = accuracy_score(y_test_trees, y_pred_opt)
    
    rfe_optimization_results.append({
        'n_features': n_features,
        'cv_f1_mean': cv_scores.mean(),
        'cv_f1_std': cv_scores.std(),
        'test_f1': test_f1,
        'test_accuracy': test_acc
    })
    
    print(f"  {n_features} features: CV F1={cv_scores.mean():.4f}±{cv_scores.std():.4f}, Test F1={test_f1:.4f}")

print()

df_rfe_opt = pd.DataFrame(rfe_optimization_results)
df_rfe_opt.to_csv('rfe_optimization_results.csv', index=False)
print("✓ Guardado: rfe_optimization_results.csv")
print()

# Mejor configuración
best_n = df_rfe_opt.loc[df_rfe_opt['test_f1'].idxmax(), 'n_features']
print(f"✓ Mejor número de features: {int(best_n)}")
print()

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(df_rfe_opt['n_features'], df_rfe_opt['test_f1'], 'o-', linewidth=2, markersize=8, label='Test F1')
plt.plot(df_rfe_opt['n_features'], df_rfe_opt['cv_f1_mean'], 's--', linewidth=2, markersize=8, label='CV F1 Mean')
plt.xlabel('Número de Features', fontsize=12)
plt.ylabel('F1-Score', fontsize=12)
plt.title('Optimización RFE - Performance vs Número de Features', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('rfe_optimization_curve.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Guardado: rfe_optimization_curve.png")
print()

# =============================================================================
# FASE 5: VALIDACIÓN REPETIDA
# =============================================================================

print("="*80)
print("FASE 5: VALIDACIÓN REPETIDA (REPEATED STRATIFIED K-FOLD)")
print("="*80)
print()

print("Configuración: 5-Fold × 3 repeticiones = 15 evaluaciones")
print("(Esto puede tardar 5-10 minutos)")
print()

# Configurar validación repetida
repeated_cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=3,
    random_state=RANDOM_STATE
)

# Scoring múltiple
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

modelos_validacion = {
    'Logistic Regression': (lr_model, X_train_logit_scaled, y_train_logit_balanced),
    'Random Forest (40f)': (rf_model, X_train_rf, y_train_trees_balanced),
    'XGBoost': (xgb_model, X_train_trees_balanced, y_train_trees_balanced),
    'Ensemble (RF+XGB)': (ensemble, X_train_rf, y_train_trees_balanced)
}

resultados_validacion = []

for model_name, (model, X_train, y_train) in modelos_validacion.items():
    print(f"Validando {model_name}...")
    
    cv_results = cross_validate(
        model, X_train, y_train,
        cv=repeated_cv,
        scoring=scoring,
        n_jobs=-1
    )
    
    resultados_validacion.append({
        'Model': model_name,
        'Accuracy_mean': cv_results['test_accuracy'].mean(),
        'Accuracy_std': cv_results['test_accuracy'].std(),
        'Precision_mean': cv_results['test_precision'].mean(),
        'Precision_std': cv_results['test_precision'].std(),
        'Recall_mean': cv_results['test_recall'].mean(),
        'Recall_std': cv_results['test_recall'].std(),
        'F1_mean': cv_results['test_f1'].mean(),
        'F1_std': cv_results['test_f1'].std(),
        'ROC_AUC_mean': cv_results['test_roc_auc'].mean(),
        'ROC_AUC_std': cv_results['test_roc_auc'].std()
    })
    
    print(f"  ✓ F1 = {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

print()

df_validacion = pd.DataFrame(resultados_validacion)

print("RESULTADOS VALIDACIÓN REPETIDA:")
print()
print(df_validacion.to_string(index=False))
print()

df_validacion.to_csv('validacion_repetida_results.csv', index=False)
print("✓ Guardado: validacion_repetida_results.csv")
print()

# Visualización de intervalos de confianza
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Validación Repetida - Intervalos de Confianza (95%)', fontsize=16, fontweight='bold')

metricas_plot = ['Accuracy', 'Precision', 'Recall', 'F1']
for idx, metric in enumerate(metricas_plot):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    means = df_validacion[f'{metric}_mean']
    stds = df_validacion[f'{metric}_std']
    models = df_validacion['Model']
    
    # Intervalos 95% (aprox 2*std)
    ci = 1.96 * stds
    
    ax.errorbar(range(len(models)), means, yerr=ci, fmt='o', markersize=8, 
                capsize=5, capthick=2, linewidth=2)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} (Mean ± 95% CI)', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3, axis='y')
    ax.set_ylim([0.7, 1.0])

plt.tight_layout()
plt.savefig('validacion_repetida_intervals.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Guardado: validacion_repetida_intervals.png")
print()

# =============================================================================
# SECCIÓN FINAL: REPORTE Y RECOMENDACIONES
# =============================================================================

print("="*80)
print("REPORTE FINAL Y RECOMENDACIONES")
print("="*80)
print()

# Determinar mejor modelo
best_model_idx = df_validacion['F1_mean'].idxmax()
best_model_name = df_validacion.loc[best_model_idx, 'Model']
best_f1_mean = df_validacion.loc[best_model_idx, 'F1_mean']
best_f1_std = df_validacion.loc[best_model_idx, 'F1_std']

print("🏆 MODELO RECOMENDADO:")
print(f"   {best_model_name}")
print(f"   F1-Score: {best_f1_mean:.4f} ± {best_f1_std:.4f}")
print()

# Comparación ensemble vs individuales
if 'Ensemble' in best_model_name:
    print("✅ ENSEMBLE SUPERIOR: Combinar RF + XGBoost mejora performance")
else:
    mejora_ensemble = df_validacion[df_validacion['Model'] == 'Ensemble (RF+XGB)']['F1_mean'].values[0]
    mejora_individual = df_validacion[df_validacion['Model'] == best_model_name]['F1_mean'].values[0]
    
    if mejora_ensemble > mejora_individual:
        print(f"⚠️ ENSEMBLE LIGERAMENTE MEJOR: {mejora_ensemble:.4f} vs {mejora_individual:.4f}")
        print("   Considerar usar ensemble para producción")
    else:
        print(f"✅ MODELO INDIVIDUAL ÓPTIMO: {best_model_name}")
        print("   Ensemble no aporta mejora significativa")

print()

# Análisis RFE
best_n_features = df_rfe_opt.loc[df_rfe_opt['test_f1'].idxmax(), 'n_features']
f1_40_features = df_rfe_opt[df_rfe_opt['n_features'] == 30]['test_f1'].values
f1_best_n = df_rfe_opt.loc[df_rfe_opt['test_f1'].idxmax(), 'test_f1']

if len(f1_40_features) > 0 and (f1_best_n - f1_40_features[0]) < 0.01:
    print(f"📊 REDUCCIÓN DE FEATURES EXITOSA:")
    print(f"   {int(best_n_features)} features mantienen performance")
    print(f"   Simplificar modelo sin perder accuracy")
else:
    print("⚠️ MANTENER 40 FEATURES:")
    print("   Reducción afecta significativamente performance")

print()

# Recomendaciones finales
print("📋 RECOMENDACIONES PARA TESIS/PAPER:")
print()
print("1. MODELO FINAL:")
print(f"   → Usar {best_model_name}")
print(f"   → F1 = {best_f1_mean:.4f} (validación cruzada repetida)")
print()

print("2. MÉTRICAS CLÍNICAS A REPORTAR:")
# Obtener métricas clínicas del mejor modelo
if 'Ensemble' in best_model_name:
    best_metrics = metricas_ensemble
else:
    best_metrics = [m for m in metricas_clinicas if m['Model'] == best_model_name.split(' (')[0]][0]

print(f"   → Sensibilidad: {best_metrics['Recall (Sensibilidad)']:.1%}")
print(f"   → Especificidad: {best_metrics['Especificidad']:.1%}")
print(f"   → PPV: {best_metrics['Precision (PPV)']:.1%}")
print(f"   → NPV: {best_metrics['NPV']:.1%}")
print()

print("3. EXPLICABILIDAD:")
if SHAP_AVAILABLE:
    print("   → Usar SHAP beeswarm plot para paper")
    print("   → Discutir top 5 variables clínicamente")
else:
    print("   → Instalar SHAP para análisis de explicabilidad")
print()

print("4. VALIDACIÓN:")
print("   → Mencionar validación repetida 5×3")
print("   → Reportar intervalos de confianza")
print("   → Robustez demostrada")
print()

# Crear reporte JSON
reporte_final = {
    'fecha': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'mejor_modelo': best_model_name,
    'f1_score': {
        'mean': float(best_f1_mean),
        'std': float(best_f1_std),
        'ci_95': [float(best_f1_mean - 1.96*best_f1_std), 
                  float(best_f1_mean + 1.96*best_f1_std)]
    },
    'metricas_clinicas': {
        'sensibilidad': float(best_metrics['Recall (Sensibilidad)']),
        'especificidad': float(best_metrics['Especificidad']),
        'ppv': float(best_metrics['Precision (PPV)']),
        'npv': float(best_metrics['NPV'])
    },
    'confusion_matrix': {
        'TP': int(best_metrics['TP']),
        'FP': int(best_metrics['FP']),
        'TN': int(best_metrics['TN']),
        'FN': int(best_metrics['FN'])
    },
    'rfe_optimizacion': {
        'mejor_n_features': int(best_n_features),
        'features_originales': 42
    }
}

with open('reporte_final_paso10.json', 'w') as f:
    json.dump(reporte_final, f, indent=2)

print("✓ Reporte guardado: reporte_final_paso10.json")
print()

# =============================================================================
# ARCHIVOS GENERADOS
# =============================================================================

print("="*80)
print("ARCHIVOS GENERADOS")
print("="*80)
print()

print("📁 MÉTRICAS Y RESULTADOS:")
print("  1. metricas_clinicas_extendidas.csv")
print("  2. ensemble_comparison.csv")
print("  3. rfe_optimization_results.csv")
print("  4. validacion_repetida_results.csv")
if SHAP_AVAILABLE:
    print("  5. shap_importance_xgboost.csv")
print()

print("📊 VISUALIZACIONES:")
if SHAP_AVAILABLE:
    print("  6. shap_xgboost_summary.png")
    print("  7. shap_xgboost_beeswarm.png")
    print("  8. shap_rf_summary.png")
print("  9. rfe_optimization_curve.png")
print("  10. validacion_repetida_intervals.png")
print()

print("📄 REPORTES:")
print("  11. reporte_final_paso10.json")
print()

print("="*80)
print("✅ PASO 10 COMPLETADO EXITOSAMENTE")
print("="*80)
print()

print("🎉 ANÁLISIS COMPLETO")
print()
print("Has completado:")
print("  ✓ Métricas clínicas extendidas")
print("  ✓ Explicabilidad (SHAP)")
print("  ✓ Ensemble optimization")
print("  ✓ RFE refinement")
print("  ✓ Validación cruzada repetida")
print()
print("Tu proyecto está listo para:")
print("  → Defensa de tesis")
print("  → Publicación científica")
print("  → Presentación a comité biomédico")
print()
print("="*80)

PASO 10: MEJORA Y VALIDACIÓN AVANZADA DE MODELOS

✓ SHAP disponible
✓ Imports completados

SECCIÓN 1: CARGA DE DATOS Y PREPARACIÓN

Cargando datasets...
Limpiando formato de datos...
Verificando tipos de datos...
✓ Dataset A limpio: (538, 42)
✓ Dataset B limpio: (538, 19)

Preparando datos...
✓ Datos preparados (split, SMOTE, escalamiento)

FASE 1: MÉTRICAS CLÍNICAS EXTENDIDAS

Entrenando modelos para evaluación...

✓ Modelos entrenados

MÉTRICAS CLÍNICAS EXTENDIDAS:

              Model  TN  FP  FN  TP  Accuracy  Precision (PPV)  Recall (Sensibilidad)  Especificidad  F1-Score      NPV      LR+      LR-
Logistic Regression  63  10   4  31  0.870370         0.756098               0.885714       0.863014  0.815789 0.940299 6.465714 0.132426
      Random Forest  66   7   6  29  0.879630         0.805556               0.828571       0.904110  0.816901 0.916667 8.640816 0.189610
            XGBoost  66   7   6  29  0.879630         0.805556               0.828571       0.904110  0.816901 0.